In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json

In [ ]:
# 1. 네이버 블로그에서 ‘뜨개카페’ 검색
# 2. 블로그 글에서 지도 위치 찍은 정보 가져옴 (위치 좌표)
# 3. 해당 정보를 가지고 DB화
# 4. DB위치 정보를 가지고 대시보드 지도에 카페 위치를 보여줌

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://section.blog.naver.com/")

### 1. 네이버 블로그에서 '뜨개카페' 검색

In [3]:
driver.find_element(
    By.CSS_SELECTOR,
    "#header > div.header_common > div > div.area_search > form > fieldset > div > input",
).send_keys("뜨개카페")
driver.find_element(
    By.CSS_SELECTOR,
    "#header > div.header_common > div > div.area_search > form > fieldset > div > input",
).send_keys(Keys.ENTER)

### 2. 블로그 글에서 지도 위치 찍은 정보 가져옴 (위치 좌표)

In [ ]:
# 1. 첫번째 블로그 주소를 가져옴
#  - id, no를 가져와서 url 연결 
# 2. if 지도 위치 selector가 있으면 DB에 저장
# 3. else 없으면 / 뒤로가기
# 4. 두번째 블로그 주소를 가져옴 
#  반복

In [ ]:
# 블로그 글 주소를 가져와서 
# split / 해서 뒤에 두개를 각각 id, no로 지정 
# 지정해서 변수에 넣은 뒤 
# driver.get으로 url연결 후 블로그 글 연결 
# open된 블로그 글에서 지도 정보 크롤링 
# 크롤링 된 지도 정보 db에 저장 

- 주소 정보 하나 가져와서 url 연결후 지도 정보를 저장

In [ ]:
#content > section > div.area_list_search > div:nth-child(1) > div > div.info_post > div.desc > a.desc_inner

In [4]:
# 첫번째 블로그 글 주소 정보를 가져옴 
elem = driver.find_element(
    By.CSS_SELECTOR, "#content > section > div.area_list_search > div:nth-child(1) > div > div.info_post > div.desc > a.desc_inner"
)

In [5]:
elem.get_attribute('href').split('/') # 주소를 / 기준으로 리스트

['https:', '', 'blog.naver.com', 'ksysysy', '223938309758']

In [28]:
elem.get_attribute('href').split('/')[-2]

'ksysysy'

In [6]:
# id, no 변수 
blogID = elem.get_attribute('href').split('/')[-2]
logNo = elem.get_attribute('href').split('/')[-1]

In [7]:
# url 연결
driver.get(f"https://blog.naver.com/PostView.naver?blogId={blogID}&logNo={logNo}")

In [ ]:
# div.se-module.se-module-map-text > a
# div.se-module.se-module-map-text > a

In [ ]:
# 지도정보 변수
map = driver.find_element(
    By.CSS_SELECTOR, "#SE-A2CD9D4B-A88C-4D4E-B694-8DF61219574F > div > div > div.se-module.se-module-map-text > a"
)

In [ ]:
map.get_attribute('data-linkdata') # 위도 경도 정보 크롤링 

'{"eventTarget" : "placeDesc", "placeId" : "1051270692", "searchEngine" : "naver", "searchType" : "s", "name" : "바늘이야기 연희점", "address" : "서울특별시 서대문구 연희로11가길 15", "latitude" : "37.5674563", "longitude" : "126.9291304", "tel" : "0507-1379-9771", "bookingUrl" : null }'

In [67]:
blog_links = []

# 5개만 반복 
blog_cards = driver.find_elements(
    By.CSS_SELECTOR, "#content > section > div.area_list_search > div"
)[:7]

# 블로그 주소, id, no 추출
for card in blog_cards:
    try:
        elem = card.find_element(By.CSS_SELECTOR, "div > div.info_post > div.desc > a.desc_inner")
        blog_links.append(elem.get_attribute("href"))
    except:
        continue


In [70]:
cafe = []

for blog_url in blog_links:
    try:
        blogID = blog_url.split("/")[-2]
        logNo = blog_url.split("/")[-1]
        # 블로그 접속
        driver.get(f"https://blog.naver.com/PostView.naver?blogId={blogID}&logNo={logNo}")
        time.sleep(2)

        try:
            # 지도 정보가 있는지 확인
            map_elem = driver.find_element(
                By.CSS_SELECTOR,
                "div.se-module.se-module-map-text > a"
            )
            map_data_raw = map_elem.get_attribute("data-linkdata")
            map_data = json.loads(map_data_raw)

            # 중복 제거 
            name_list = [c['카페 이름'] for c in cafe]
            if map_data["name"] in name_list:
                print(f"중복: {map_data['name']} → 건너뜀")
                continue

            cafe.append({
                
                "카페 이름": map_data["name"],
                "카페 주소": map_data["address"],
                "위도": map_data["latitude"],
                "경도": map_data["longitude"],
            })
            
        except:
            print("지도정보 없음")

    except Exception as e:
        print("오류 발생:", e)
        continue


중복: 바늘이야기 연희점 → 건너뜀
지도정보 없음
중복: 메리노 → 건너뜀


In [71]:
cafe

[{'카페 이름': '바늘이야기 연희점',
  '카페 주소': '서울특별시 서대문구 연희로11가길 15',
  '위도': '37.5674563',
  '경도': '126.9291304'},
 {'카페 이름': '메리노',
  '카페 주소': '서울특별시 성동구 서울숲길 24 1층',
  '위도': '37.5478772',
  '경도': '127.0400156'},
 {'카페 이름': '누뗀',
  '카페 주소': '서울특별시 서초구 청계산로 119 지하1층~2층',
  '위도': '37.4537511',
  '경도': '127.05127'},
 {'카페 이름': '제주포슬',
  '카페 주소': '제주특별자치도 서귀포시 대정읍 무영로228번길 17-7 제주포슬',
  '위도': '33.2811244',
  '경도': '126.2235554'}]

In [ ]:
# 한페이지에 7개 
# 그러면 70개 정도만 가져오기 -> 페이지 10페이지까지 반복문 

# cafe 정보를 가져오는 도중에 중복 제거해서 리스트에 추가 

# 아마 최종 코드

In [3]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://section.blog.naver.com/")

driver.find_element(
    By.CSS_SELECTOR,
    "#header > div.header_common > div > div.area_search > form > fieldset > div > input",
).send_keys("뜨개카페")
driver.find_element(
    By.CSS_SELECTOR,
    "#header > div.header_common > div > div.area_search > form > fieldset > div > input",
).send_keys(Keys.ENTER)

In [4]:
# 페이지 반복 
cafe = []
for i in range(1,12):
    url = f"https://section.blog.naver.com/Search/Post.naver?pageNo={i}&rangeType=ALL&orderBy=sim&keyword=%EB%9C%A8%EA%B0%9C%EC%B9%B4%ED%8E%98"
    driver.get(url)
    time.sleep(2)
    
    blog_links = []

    # 한페이지에 7개 블로그 글 
    blog_cards = driver.find_elements(
        By.CSS_SELECTOR, "#content > section > div.area_list_search > div"
    )[:7]

    # 블로그 주소, id, no 추출
    for card in blog_cards:
        try:
            elem = card.find_element(By.CSS_SELECTOR, "div > div.info_post > div.desc > a.desc_inner")
            blog_links.append(elem.get_attribute("href"))
        except:
            continue
    

    for blog_url in blog_links:
        try:
            blogID = blog_url.split("/")[-2]
            logNo = blog_url.split("/")[-1]
            # 블로그 접속
            driver.get(f"https://blog.naver.com/PostView.naver?blogId={blogID}&logNo={logNo}")
            time.sleep(2)

            try:
                # 지도 정보가 있는지 확인
                map_elem = driver.find_element(
                    By.CSS_SELECTOR,
                    "div.se-module.se-module-map-text > a"
                )
                map_data_raw = map_elem.get_attribute("data-linkdata")
                map_data = json.loads(map_data_raw)
                
                name_list = [c['카페 이름'] for c in cafe]
                if map_data["name"] in name_list:
                    print(f"중복: {map_data['name']} → 건너뜀")
                    continue


                cafe.append({
                    "카페 이름": map_data["name"],
                    "카페 주소": map_data["address"],
                    "위도": map_data["latitude"],
                    "경도": map_data["longitude"],
                })
                
            except:
                print("지도정보 없음")

        except Exception as e:
            print("오류 발생:", e)
            continue



중복: 바늘이야기 연희점 → 건너뜀
지도정보 없음
중복: 메리노 → 건너뜀
중복: 바늘이야기 연희점 → 건너뜀
중복: 바늘이야기 연희점 → 건너뜀
중복: 바늘이야기 파주직영점 → 건너뜀
중복: 누뗀 → 건너뜀
중복: 바늘이야기 연희점 → 건너뜀
중복: 바늘이야기 연희점 → 건너뜀
지도정보 없음
지도정보 없음
중복: 바늘이야기 연희점 → 건너뜀
중복: 누뗀 → 건너뜀
지도정보 없음
중복: 누뗀 → 건너뜀
중복: 바늘이야기 파주직영점 → 건너뜀
지도정보 없음
중복: 누뗀 → 건너뜀
중복: 바늘이야기 연희점 → 건너뜀
중복: 누뗀 → 건너뜀
중복: 누뗀 → 건너뜀
중복: 누뗀 → 건너뜀
중복: 옐로우헤이트 → 건너뜀
중복: 바늘이야기 연희점 → 건너뜀
중복: 바늘이야기 파주직영점 → 건너뜀
중복: 누뗀 → 건너뜀
중복: 옐로우헤이트 → 건너뜀
중복: 메리노 → 건너뜀
중복: 누뗀 → 건너뜀
지도정보 없음
중복: 솜솜뜨개 → 건너뜀
중복: 누뗀 → 건너뜀
중복: 옐로우헤이트 → 건너뜀
중복: 누뗀 → 건너뜀
중복: 누뗀 → 건너뜀
지도정보 없음
중복: 누뗀 → 건너뜀
중복: 옐로우헤이트 → 건너뜀
중복: 누뗀 → 건너뜀
지도정보 없음
중복: 누뗀 → 건너뜀
중복: 누뗀 → 건너뜀
중복: 누뗀 → 건너뜀
중복: 누뗀 → 건너뜀
중복: 누뗀 → 건너뜀
지도정보 없음
중복: 바늘이야기 파주직영점 → 건너뜀
지도정보 없음
중복: 메리노 → 건너뜀
지도정보 없음
지도정보 없음
중복: 메리노 → 건너뜀
중복: 누뗀 → 건너뜀
지도정보 없음
중복: 누뗀 → 건너뜀
중복: 카페토요 → 건너뜀


In [5]:
len(cafe)

21

In [6]:
cafe

[{'카페 이름': '바늘이야기 연희점',
  '카페 주소': '서울특별시 서대문구 연희로11가길 15',
  '위도': '37.5674563',
  '경도': '126.9291304'},
 {'카페 이름': '메리노',
  '카페 주소': '서울특별시 성동구 서울숲길 24 1층',
  '위도': '37.5478772',
  '경도': '127.0400156'},
 {'카페 이름': '누뗀',
  '카페 주소': '서울특별시 서초구 청계산로 119 지하1층~2층',
  '위도': '37.4537511',
  '경도': '127.05127'},
 {'카페 이름': '제주포슬',
  '카페 주소': '제주특별자치도 서귀포시 대정읍 무영로228번길 17-7 제주포슬',
  '위도': '33.2811244',
  '경도': '126.2235554'},
 {'카페 이름': '바늘이야기 파주직영점',
  '카페 주소': '경기도 파주시 탄현면 법흥로 100-1',
  '위도': '37.7765751',
  '경도': '126.706411'},
 {'카페 이름': '카페토요',
  '카페 주소': '서울특별시 도봉구 시루봉로 149 102호',
  '위도': '37.6642895',
  '경도': '127.0282775'},
 {'카페 이름': '바늘카페',
  '카페 주소': '서울특별시 서대문구 연희로11가길 15 2층',
  '위도': '37.5674563',
  '경도': '126.9291304'},
 {'카페 이름': '카페 슬로우플로우',
  '카페 주소': '경기도 김포시 김포한강11로140번길 116 슬로우플로우 1층 카페',
  '위도': '37.6475742',
  '경도': '126.6819604'},
 {'카페 이름': '옐로우헤이트',
  '카페 주소': '서울특별시 관악구 행운6길 22 1층',
  '위도': '37.4806707',
  '경도': '126.9609454'},
 {'카페 이름': '에이하 커피',
  '카페 주소': '경기도 의왕시

In [32]:
import pandas as pd
df_cafe = pd.DataFrame(cafe)
df_cafe.head(5)

,카페 이름,카페 주소,위도,경도
0,바늘이야기 연희점,서울특별시 서대문구 연희로11가길 15,37.5674563,126.9291304
1,메리노,서울특별시 성동구 서울숲길 24 1층,37.5478772,127.0400156
2,누뗀,서울특별시 서초구 청계산로 119 지하1층~2층,37.4537511,127.05127
3,제주포슬,제주특별자치도 서귀포시 대정읍 무영로228번길 17-7 제주포슬,33.2811244,126.2235554
4,바늘이야기 파주직영점,경기도 파주시 탄현면 법흥로 100-1,37.7765751,126.706411


In [ ]:
# 블로그 내용 크롤링
# 작성자, 작성일, 블로그 내용

In [4]:
# 블로그 내용 크롤링
# url 연결
driver.get("https://blog.naver.com/PostView.naver?blogId=ksysysy&logNo=223938309758")

In [5]:
blog = driver.find_element(By.CSS_SELECTOR, "#printPost1 > tbody > tr > td.bcc").text


In [6]:
blog

'제1의 취미 여행\n[바늘이야기 연희점] 뜨개질 전문 매장, 뜨개 카페.\n전통보리차 ・ 2025. 7. 18. 13:55\nURL 복사 이웃추가\n본문 기타 기능\n위치바늘이야기 연희점\n6/12.\n뜨개(수집)인으로서 예전부터 꼭 가보고싶었던\n바늘이야기 카페에 다녀왔다.\n홍대 나들이 나간김에 연희점으로 다녀왔는데\n파주점도 가보고싶긴하다.\n바늘이야기 연희점.\n4층 건물을 통째로 쓰고있다.\n뜨개용품 샵과 스튜디오, 카페도 운영하고 있어서\n뜨개인들에게는 그야말로 성지인데\n아쉽게도 주차장이 없다.\n심지어 이 주변에 주차장 찾기도 너무 어려웠다.\n들어서자마자 한 면이 실로 꽉 채워진 인테리어.\n뜨개인들에게는 그야말로 최고의 인테리어되겠다.\n완성된 뜨개 제품도 판매하고 있다.\n이 작품을 뜨기 위해 사용된 실과 도안, 소요량까지\n태그 달려있어서 좋다.\n물론. 저는 못뜨지만요. ㅋ\n손으로 직접 실의 질감과 재질도 느껴보고\n구매할 수 있다는게 가장 매력적이었다.\n벽면에 가득한 다양한 실 구경이 가장 재미있었다.\n뜨개초보라서 대체 무슨실을 사야할 지를 모르는게\n가장 아쉬웠지만.\n고급스러운 캐시미어실부터\n조금더 쉽게 뜨개질에 입문할 수 있는\n수세미실과 면실까지 종류와 컬러가 다양했다.\n다양한 대바늘 스와치♡\n나 이거 언제 다 가능하게 되는걸까.\n다양한 바늘 종류들도 구매 가능하다.\n코바늘에 빠져서 코바늘은 많이 들였는데..\n이제 대바늘 차례인가요.\n종류가 무궁무진해서 힘들구나.\n눈돌아가는 다양한 부자재들.\n우리도 여기까지 갔으니 무언가 하나 사서\n2층 카페에 올라가 바늘을 잡아보기로 했다.\n간단한 키링 만들기 키트부터\n작은 소품과 가방까지 만들 수 있는 diy키트도 있다.\n코바늘과 대바늘에 입문할 수 있는 세트도 판매.\n실과 바늘, 도안을 따로따로 구매하는 것보다\n가격대가 있는 편이었지만\n대바늘을 시작해볼까 하는 마음에 구매했다.\n대바늘 키트를 구매헤서 2층 카페로 올라갔다.\n자리가 많은 편인데도 사람이

In [ ]:
# 작성일 가져오기 


## 최최종코드 

In [34]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://section.blog.naver.com/")

driver.find_element(
    By.CSS_SELECTOR,
    "#header > div.header_common > div > div.area_search > form > fieldset > div > input",
).send_keys("뜨개카페")
driver.find_element(
    By.CSS_SELECTOR,
    "#header > div.header_common > div > div.area_search > form > fieldset > div > input",
).send_keys(Keys.ENTER)

In [ ]:
# 페이지 반복 
cafe = []
for i in range(1,12):
    url = f"https://section.blog.naver.com/Search/Post.naver?pageNo={i}&rangeType=ALL&orderBy=recentdate&keyword=%EB%9C%A8%EA%B0%9C%EC%B9%B4%ED%8E%98"
    driver.get(url)
    time.sleep(2)
    
    blog_links = []

    # 한페이지에 7개 블로그 글 
    blog_cards = driver.find_elements(
        By.CSS_SELECTOR, "#content > section > div.area_list_search > div"
    )[:7]

    # 블로그 주소, id, no 추출
    for card in blog_cards:
        try:
            elem = card.find_element(By.CSS_SELECTOR, "div > div.info_post > div.desc > a.desc_inner")
            blog_links.append(elem.get_attribute("href"))
        except:
            continue
    

    for blog_url in blog_links:
        try:
            blogID = blog_url.split("/")[-2]
            logNo = blog_url.split("/")[-1]
            # 블로그 접속
            driver.get(f"https://blog.naver.com/PostView.naver?blogId={blogID}&logNo={logNo}")
            time.sleep(2)

            try:
                # 지도 정보가 있는지 확인
                map_elem = driver.find_element(
                    By.CSS_SELECTOR,
                    "div.se-module.se-module-map-text > a"
                )
                map_data_raw = map_elem.get_attribute("data-linkdata")
                map_data = json.loads(map_data_raw)
                # 블로그 내용 크롤링
                blog = driver.find_element(By.CSS_SELECTOR, "#printPost1 > tbody > tr > td.bcc").text

                cafe.append({
                    "카페 이름": map_data["name"],
                    "카페 주소": map_data["address"],
                    "위도": map_data["latitude"],
                    "경도": map_data["longitude"],
                    "블로그내용" : blog
                })
                
            except:
                print("지도정보 없음")

        except Exception as e:
            print("오류 발생:", e)
            continue



In [ ]:
df_cafe = pd.DataFrame(cafe)

In [ ]:
df_cafe.head(5)

In [56]:
df_cafe['블로그내용'][10]

'먹고싶다\n[파주/탄현면] 뜨개러라면 눈이 휘둥그래질, 뜨개카페 바늘이야기\n뜨다만인기녀 ・ 2024. 11. 9. 21:28\nURL 복사 이웃추가\n본문 기타 기능\n위치바늘이야기 파주직영점\n#파주뜨개카페 #파주카페 #바늘이야기 #바늘이야기파주점 #내돈내산 #뜨개질 #뜨개실\n나의 오랜 취미는 가죽 공예였는데 공간의 제약이 너무나도 커서 다른 취미를 찾아봐야 겠다는 생각을 했다.\n아무래도 손으로 사부작 거리는 것을 좋아하는 나이기에 뜨개질을 배워보면 어떨까 싶었고, 잠깐 배운적인 있는 대바늘 말고 코바늘로 도전 시작!\n새복이 유치원 친구 엄마들 대부분이 뜨개질 능력자셔서 덕분에 무료로(!) 배우고 있다.\n가장 처음으로 실버백을 도전해봤는데 와 이실 쉽지 않음....\n바늘 코 넣을 곳이 눈에 잘 안보여서 초보가 고르면 안되는 실이었음을....\n어쨌든 어찌저치 실 하나를 다 쓰고 다음은 뭘 해 볼까 고민하다가 급 바늘이야기 카페로 향했다 :)\n과거에 실공장1998이었고 지금은 바늘이야기인 공간.\n예전에도 좋았지만 지금이 더 좋다고 느낄 정도로 공간 구성이 잘되어있었다.\n일단 실도 많고, 샘플도 많아서 구경거리가 많았다.\n예전에는 카페가 메인이라고 느껴졌다면 지금은 뜨개가 메인인 느낌?\n1층은 뜨개용품을 팔고 2,3층은 카페로 운영되고 있는데, 비오는 평일 오전에 가서일까 매장이 한적해서 좋았다.\n(벽면에 실을 그라데이션으로 배치한 인테리어도 예뻤음)\n뜨개 초보니까 일단 샘플부터 살펴보았다.\n아직 무것을 떠볼까 결정하지 못하고 방문했기 때문이다.\n참고로 가방을 참 좋아하지만 소품은 더더더욱 좋아하는 편이다.\n그래서 아기자기한 소품류에 눈을 뗄 수가 없었다.\n옷 샘플도 참 많던데 저에겐 그림의 떡이구요^^\n실이 정말정말 많았다.\n온라인으로 사면 실의 촉감이나 색감을 직접 보지 못해서 아쉬운데 이렇게 쇼룸같은 카페가 있으니 얼마나 좋은지!\n하지만....\n내가 사고 싶은 실은 이 곳에 재고가 없었다.\n방문 당시만 해도 방문자들

In [ ]:
df_cafe['블로그내용'][0].split('\n')[2].split('・')

['전통보리차 ', ' 2025. 7. 18. 13:55']

In [ ]:
작성자 / 작성일 